
# 🦥 FineTuning With Unsloth
## In this tutorial we'll finetune the Llama-3.2-3B-Instruct model using unsloth on the ServiceNow-AI/R1-Distill-SFT dataset to empower the llama model with DeepSeek-R1 like 'thinking' capabilities.

Let's start with installing the dependencies.


In [1]:
!curl -LsSf https://astral.sh/uv/install.sh | sh
!pip install uv

downloading uv 0.9.18 x86_64-unknown-linux-gnu
no checksums to verify
installing to /usr/local/bin
  uv
  uvx
everything's installed!
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 22.7 MB/s eta 0:00:00


In [2]:
"""Llama 3.2 Finetuning with Unsloth"""

# Step 1: Install dependencies
!curl -LsSf https://astral.sh/uv/install.sh | sh
!pip install uv
!uv pip install --system Pillow==10.2.0 psutil datasets transformers
!uv pip install --system "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!uv pip install --system --no-deps trl peft accelerate bitsandbytes xformers

downloading uv 0.9.18 x86_64-unknown-linux-gnu
no checksums to verify
installing to /usr/local/bin
  uv
  uvx
everything's installed!
Using Python 3.12.12 environment at: /usr
Resolved 38 packages in 543ms
Prepared 1 package in 93ms
Uninstalled 1 package in 4ms
Installed 1 package in 2ms
 - pillow==11.3.0
 + pillow==10.2.0
Using Python 3.12.12 environment at: /usr
Resolved 81 packages in 4.32s
Prepared 10 packages in 1.92s
Uninstalled 3 packages in 157ms
Installed 10 packages in 28ms
 + bitsandbytes==0.49.0
 + cut-cross-entropy==25.1.1
 - datasets==4.0.0
 + datasets==4.3.0
 + msgspec==0.20.0
 - pyarrow==18.1.0
 + pyarrow==22.0.0
 - torchao==0.10.0
 + torchao==0.15.0
 + trl==0.24.0
 + tyro==1.0.3
 + unsloth==2025.12.9 (from git+https://github.com/unslothai/unsloth.git@21d897d131ffb56b9b577e0652f1012404586c99)
 + unsloth-zoo==2025.12.7
Using Python 3.12.12 environment at: /usr
Resolved 5 packages in 33ms
Prepared 1 package in 1.45s
Installed 1 package in 4ms
 + xformers==0.0.33.post2


In [1]:
# Step 1: Unsloth
import psutil

# Unsloth compiled cache psutil inject
import sys
import builtins

# Global namespace in psutil add
builtins.psutil = psutil
globals()['psutil'] = psutil

In [2]:
import unsloth
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/usr/local/lib/python3.12/dist-packages/torchao/quantization/quant_api.py:2525: SyntaxWarning: invalid escape sequence '\.'
  * regex for parameter names, must start with `re:`, e.g. `re:language\.layers\..+\.q_proj.weight`.


🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
# Model configuration
max_seq_length = 2048
dtype = None
load_in_4bit = True

In [4]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.12.9: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.12.9 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [6]:
dataset = load_dataset("ServiceNow-AI/R1-Distill-SFT", 'v0', split="train")
print(f"Dataset size: {len(dataset)}")

README.md: 0.00B [00:00, ?B/s]

v0/train-00000-of-00003.parquet:   0%|          | 0.00/180M [00:00<?, ?B/s]

v0/train-00001-of-00003.parquet:   0%|          | 0.00/187M [00:00<?, ?B/s]

v0/train-00002-of-00003.parquet:   0%|          | 0.00/188M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/171647 [00:00<?, ? examples/s]

Dataset size: 171647


In [9]:
dataset

Dataset({
    features: ['id', 'reannotated_assistant_content', 'problem', 'source', 'solution', 'verified', 'quality_metrics'],
    num_rows: 171647
})

In [10]:
# Prepare prompt template
r1_prompt = """You are a reflective assistant engaging in thorough, iterative reasoning, mimicking human stream-of-consciousness thinking. Your approach emphasizes exploration, self-doubt, and continuous refinement before coming up with an answer.
<problem>
{}
</problem>

{}
{}
"""
EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    problems = examples["problem"]
    thoughts = examples["reannotated_assistant_content"]
    solutions = examples["solution"]
    texts = []

    for problem, thought, solution in zip(problems, thoughts, solutions):
        text = r1_prompt.format(problem, thought, solution) + EOS_TOKEN
        texts.append(text)

    return {"text": texts}

In [11]:
dataset = dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/171647 [00:00<?, ? examples/s]

In [12]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/171647 [00:00<?, ? examples/s]

In [13]:
trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 171,647 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856 of 3,237,063,680 (0.75% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.010000
2,0.935000
3,1.034000
4,0.944000
5,0.785900
6,0.853600
7,0.756500
8,0.741600
9,0.786500
10,0.736800


TrainOutput(global_step=60, training_loss=0.6270231356223425, metrics={'train_runtime': 566.8205, 'train_samples_per_second': 0.847, 'train_steps_per_second': 0.106, 'total_flos': 8307288499716096.0, 'train_loss': 0.6270231356223425, 'epoch': 0.002796420581655481})

In [14]:
from unsloth.chat_templates import get_chat_template
sys_prompt = """You are a reflective assistant engaging in thorough, iterative reasoning, mimicking human stream-of-consciousness thinking. Your approach emphasizes exploration, self-doubt, and continuous refinement before coming up with an answer.
<problem>
{}
</problem>
"""
message = sys_prompt.format("How many 'r's are present in 'strawberry'?")
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": message},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 1024, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
response = tokenizer.batch_decode(outputs)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [15]:
print(response[0])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

You are a reflective assistant engaging in thorough, iterative reasoning, mimicking human stream-of-consciousness thinking. Your approach emphasizes exploration, self-doubt, and continuous refinement before coming up with an answer.
<problem>
How many 'r's are present in'strawberry'?
</problem>
<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Alright, let me figure out how many 'r's are in the word'strawberry'. Okay, so I remember that 'r' is the seventh letter of the alphabet, but is that correct? Let me just check again to make sure. In the standard English alphabet, 'r' is indeed the seventh letter, so that's probably correct.

Okay, now, looking at'strawberry'. It's a seven-letter word. I need to count the number of 'r's in it. Let's break it down:

- S: That's's'. No 'r's here.
- T: That's 't'. Stil

In [16]:
model.save_pretrained("chintan-001-3B")  # Local saving
tokenizer.save_pretrained("chintan-001-3B")

('chintan-001-3B/tokenizer_config.json',
 'chintan-001-3B/special_tokens_map.json',
 'chintan-001-3B/chat_template.jinja',
 'chintan-001-3B/tokenizer.json')

In [17]:
model.save_pretrained_gguf("chintan-001-3B-GGUF", tokenizer,)

Unsloth: Merging model weights to 16-bit format...


config.json:   0%|          | 0.00/890 [00:00<?, ?B/s]

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Checking cache directory for required files...
Cache check failed: model-00001-of-00002.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


...

Unsloth: Preparing safetensor model files: 100%|██████████| 2/2 [06:43<00:00, 201.94s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████| 2/2 [04:36<00:00, 138.08s/it]


Unsloth: Merge process complete. Saved to `/content/chintan-001-3B-GGUF`
Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF f16 might take 3 minutes.
\        /    [2] Converting GGUF f16 to ['q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: Updating system package directories
Unsloth: All required system packages already installed!
Unsloth: Install llama.cpp and building - please wait 1 to 3 minutes
Unsloth: Cloning llama.cpp repository
Unsloth: Install GGUF and other packages
Unsloth: Successfully installed llama.cpp!
Unsloth: Preparing converter script...
Unsloth: [1] Converting model into f16 GGUF format.
This might take 3 minutes...
Unsloth: Initial conversion completed! Files: ['llama-3.2-3b-instruct.F16.gguf']


{'save_directory': 'chintan-001-3B-GGUF',
 'gguf_files': ['llama-3.2-3b-instruct.Q8_0.gguf'],
 'modelfile_location': '/content/Modelfile',
 'want_full_precision': False,
 'is_vlm': False,
 'fix_bos_token': False}